In [7]:
import requests
import io
import numpy as np
import os

#Loads data from url
def make_request(url):
    print("Requesting data from {}...".format(url))
    response = requests.get('https://content.codecademy.com/courses/deeplearning-with-tensorflow/'+url)
    response.raise_for_status()
    response_data = io.BytesIO(response.content)
    return response_data
    
#Loads galaxy data
def load_galaxy_data():
  
  #If cached file not found, loads data from url
  if not os.path.isfile('./cached_data.npz'):
     response_data = make_request(url='galaxydata.npz')

     with open("cached_data.npz","wb") as save_file:
      save_file.write(response_data.read())
 
  #Load data using NumPy
  data = np.load('cached_data.npz')

  print("Successfully loaded galaxy data!")
  
  return data["data"],data["labels"]

In [17]:
import tensorflow as tf
from matplotlib import pyplot as plt

#Visualizes convolutional layer activations
def visualize_activations(model, validation_iterator):

  #A keras model that will output our previous model's activations for each convolutional layer:
  activation_extractor = tf.keras.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers if "conv2d" in layer.name])

  #Take matplotlib frame and remove axes.
  def clean_plot(plot):
    plot.axes.get_xaxis().set_visible(False)
    plot.axes.get_yaxis().set_visible(False)

  #Dict mapping from class numbers to string labels:
  class_names = {0:"Regular",1:"Ringed",2:"Merger",3:"Other"}

  #Loads a sample batch of data
  sample_batch_input,sample_labels = validation_iterator.next()
 
  #Grabs the first five images
  sample_batch_input = sample_batch_input[:5]
  sample_labels = sample_labels[:5]

  #Makes predictions using model.predict(x)
  sample_predictions = model.predict(sample_batch_input)

  #Iterate of images, predictions, and true labels
  for i,(image, prediction, label) in enumerate(zip(sample_batch_input, sample_predictions, sample_labels)):

    image_name = "Galaxy_{}".format(i)

    #Gets predicted class with highest probability

    predicted_class = tf.argmax(prediction).numpy()

    #Gets correct label
    actual_class = tf.argmax(label).numpy()

    print(image_name)
    print("\tModel prediction: {}".format(prediction))
    print("\tTrue label: {} ({})".format(class_names[actual_class], actual_class))
    print("\tCorrect:", predicted_class == actual_class)

    #Saves image file using matplotlib
    sample_image = image
    clean_plot(plt.imshow(sample_image))

    plt.title(image_name+" Predicted: {}, Actual: {}".format(class_names[predicted_class], class_names[actual_class]))
    plt.savefig('static/images/'+image_name+".png")
    model_layer_output = activation_extractor(tf.expand_dims(sample_image,0))
    
    plt.clf()

    #Iterates over each layer output
    for l_num,output_data in enumerate(model_layer_output):

      #Creates a subplot for each filter
      fig, axs = plt.subplots(1, output_data.shape[-1])
      
      #For each filter
      for i in range(output_data.shape[-1]):

        #Plots the filter's activations
        
        clean_plot(axs[i].imshow(output_data[0][:, :, i], cmap="gray"))
      plt.suptitle(image_name+" Conv {}".format(l_num),y=0.6)
      plt.savefig('static/images/' + image_name+ "Conv{}.png".format(l_num))
      plt.clf()

In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# from utils import load_galaxy_data





input_data, labels = load_galaxy_data()

print(input_data.shape, labels.shape)

Successfully loaded galaxy data!
(1400, 128, 128, 3) (1400, 4)


In [25]:
x_train, x_test, y_train, y_test = train_test_split(input_data, labels, test_size=0.2, random_state=222, stratify=labels, shuffle=True)

In [26]:
generator = ImageDataGenerator(rescale=1.0/128)

train_iterator = generator.flow(x_train, y_train, batch_size = 5)
test_iterator = generator.flow(x_test, y_test, batch_size = 5)

In [27]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(128,128,3)))
model.add(tf.keras.layers.Conv2D(8, 3, strides=2, activation='relu', padding='valid'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2))
model.add(tf.keras.layers.Conv2D(5, 4, strides=2, activation='relu', padding='valid'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=.005), loss=[tf.keras.losses.CategoricalCrossentropy()], metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 63, 63, 8)         224       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 31, 31, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 14, 14, 5)         645       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 7, 7, 5)          0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 245)               0         
                                                                 
 dense_10 (Dense)            (None, 64)               

In [34]:
batch_size = 5
model.fit(train_iterator, steps_per_epoch=len(x_train)/batch_size, epochs=35, validation_data=test_iterator, validation_steps=len(x_test)/batch_size)

Epoch 1/35
224/224 [==============================] - 2s 7ms/step - loss: 0.1452 - categorical_accuracy: 0.9536 - auc_5: 0.9959 - val_loss: 1.1134 - val_categorical_accuracy: 0.7321 - val_auc_5: 0.8934
Epoch 2/35
224/224 [==============================] - 1s 6ms/step - loss: 0.1329 - categorical_accuracy: 0.9536 - auc_5: 0.9970 - val_loss: 1.1313 - val_categorical_accuracy: 0.7357 - val_auc_5: 0.8904
Epoch 3/35
224/224 [==============================] - 1s 6ms/step - loss: 0.1104 - categorical_accuracy: 0.9616 - auc_5: 0.9980 - val_loss: 1.2429 - val_categorical_accuracy: 0.7036 - val_auc_5: 0.8811
Epoch 4/35
224/224 [==============================] - 1s 6ms/step - loss: 0.1009 - categorical_accuracy: 0.9714 - auc_5: 0.9982 - val_loss: 1.1745 - val_categorical_accuracy: 0.7143 - val_auc_5: 0.8936
Epoch 5/35
224/224 [==============================] - 1s 6ms/step - loss: 0.0982 - categorical_accuracy: 0.9705 - auc_5: 0.9986 - val_loss: 1.1775 - val_categorical_accuracy: 0.7179 - val_auc_

In [35]:
visualize_activations(model, test_iterator)

1/1 [==============================] - 0s 22ms/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Galaxy_0
	Model prediction: [6.1706245e-01 3.7997067e-01 2.9298768e-03 3.6989913e-05]
	True label: Regular (0)
	Correct: True


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Galaxy_1
	Model prediction: [5.4908207e-09 9.9999928e-01 2.9993763e-09 7.6622518e-07]
	True label: Ringed (1)
	Correct: True


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>